In [222]:
import numpy as np
import pandas as pd
import scipy as sp
import resampy as rs 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa as tsa 
import statsmodels.formula.api as smf
import os
import glob
import random
import string

In [223]:
study_directory = '/Users/alexastefanko/lab/FoGdetection/Subjects'
output_path = '/Users/alexastefanko/lab/FoGdetection/Output/fogs.xlsx'
sample_rate = 128
step = 200

r_acc_ap_col = "R_acc"; #Column name for right accelerometer anterior-posterior
l_acc_ap_col = "L_acc"; #Column name for left accelerometer anterior-posterior
r_gyr_ml_col = "R_gyr"; #Column name for right gyroscope mediolateral
l_gyr_ml_col = "L_gyr"; #Column name for left gyroscop mediolateral

In [224]:
def get_csv_files(directory):
    for dir_name, subdir_list, file_list in os.walk(directory):
        return [os.path.join(dir_name, name) for name in file_list if name.endswith(".csv")] 
        
def get_starter_idx(data):
    r_ml = data[0]
    mean_r = np.mean(r_ml)
    r_for_cov = [x - mean_r for x in r_ml]
    
    l_ml = data[1]
    mean_l = np.mean(l_ml)
    l_for_cov = [x - mean_l for x in l_ml]
    
    #R and L angular velocities cross-correlation to find a delay between two
    cross_cov = np.correlate(r_for_cov, l_for_cov, "full")
    shifter = 0
    for idx, num in enumerate(cross_cov):
        if num == np.max(cross_cov):
            shifter = idx
    starter = round(abs(shifter - (len(cross_cov)/2)))
    return starter

def create_corr_log(corr_data, threshold):
    log = [None] * len(corr_data)
    idx = 0
    for i in corr_data:
        if i < threshold:
            log[idx] = 1
        else:
            log[idx] = 0
        idx = idx + 1
    return log
    
def correlation_based_method(starter, filtered_data):
    #Sync R and L after correction for delay
    t_sensor_1 = filtered_data[0][starter:]
    t_sensor_2 = filtered_data[1][:len(filtered_data[1]) - starter]

    correlated_sensor_data = []
    
    #Ensure bouts have sufficient data
    range_intervals = list(range(0, len(t_sensor_1) - (step+1), step))
    if len(t_sensor_1) > (step*5):
        for i in range(len(range_intervals) - 1):
            start = range_intervals[i]
            end = range_intervals[i+1]
            corr = np.corrcoef(t_sensor_1[start:end], t_sensor_2[start:end])
            correlated_sensor_data.append(corr[0][1])
        abs_corr_sensor_data = np.absolute(correlated_sensor_data)
        corr_log = create_corr_log(abs_corr_sensor_data, 0.5)
        return corr_log
    else:
        raise ValueError("Error: There is not enough data to calculate FOG. Please ensure each bout is longer than 10 seconds or sample more often")

def fft_method(starter, data, len_data):
    corrected_r_ap = data[0][starter:]
    corrected_l_ap = data[1][:len_data - starter]
    if len(corrected_r_ap) > step*5:
        range_intervals = list(range(0, len(corrected_r_ap) - (step+1), step))
        ratio_r = []
        ratio_l = []
        for i in range(len(range_intervals) - 1):
            start = range_intervals[i]
            end = range_intervals[i+1]
            l = len(corrected_r_ap[start:end])
            list_step = int(round(l/step))
            f = list(range(0, (list_step*(l)), list_step))
            lf = f.index(3)
            hf = f.index(10)
            llf = f.index(0)
            detrend_r = sp.signal.detrend(corrected_r_ap[start:end])
            detrend_l = sp.signal.detrend(corrected_l_ap[start:end])
            p_r = abs(np.fft.fft(detrend_r))/(l/2)
            p_l = abs(np.fft.fft(detrend_l))/(l/2)
            ratio_r.append(sum(p_r[lf:hf+1])**2/sum(p_r[llf:lf+1])**2)
            ratio_l.append(sum(p_l[lf:hf+1])**2/sum(p_l[llf:lf+1])**2)
        perc = [None] * len(ratio_r)
        for j in range(len(ratio_r)):
            # Threshold on frequency ratio based on FFT
            if ratio_r[j]>10 or ratio_l[j] > 10:
                perc[j] = 1
            else:
                perc[j] = 0
    else:
        raise ValueError("Error: There is not enough data to calculate FOG. Please ensure each bout is longer than 10 seconds or sample more often")
    return perc

def find_log_overlap(corr_log, fft_log):
    final_log = [None] * len(corr_log)
    for i in range(len(corr_log)):
        if corr_log[i] == 1 and fft_log[i] == 1:
            final_log[i] = 1
        else:
            final_log[i] = 0
    return final_log

def merge_one_two_second(log):
    #this log should be its own data struct
    merged_log = log 
    episodes = [i for i, x in enumerate(log) if x == 1]
    episodes_diff = np.diff(episodes)
    indices_diff_1 = [i for i, x in enumerate(episodes_diff) if x == 2]
    indices_diff_2 = [i for i, x in enumerate(episodes_diff) if x == 3]
    for idx in indices_diff_1: 
        merged_log[episodes[idx]+1] = 1 
    for idx_2 in indices_diff_2:
        merged_log[episodes[idx_2]+1] = 1
        merged_log[episodes[idx_2]+2] = 1
    return merged_log

In [225]:
subject_files = get_csv_files(study_directory)
dataframes = [pd.read_csv(file) for file in subject_files]

In [227]:
x = 0
fog_df = pd.DataFrame(columns = ["filename", "NN", "MM", "very short", 
                                 "short", "long", "very long", "total"])
for df in dataframes:
    filename = os.path.basename(os.path.normpath(subject_files[x]))
    
    r_acc_ap = df[r_acc_ap_col].values
    l_acc_ap = df[l_acc_ap_col].values
    r_gyr_ml = df[r_gyr_ml_col].values
    l_gyr_ml = df[l_gyr_ml_col].values
    
    #Resample the data to 200 
    r_ml = rs.resample(r_gyr_ml, sample_rate, step)
    l_ml = rs.resample(l_gyr_ml, sample_rate, step)
    r_ap = rs.resample(r_acc_ap, sample_rate, step)
    l_ap = rs.resample(l_acc_ap, sample_rate, step)
    
    #apply low-pass butterworth filter
    b,a = sp.signal.butter(4,5/(200/2))
    sensor_data_filtered=sp.signal.filtfilt(b,a,[r_ml, l_ml], padtype = 'odd', padlen=3*(max(len(b),len(a))-1));
    
    #correlation-based method
    starter_idx = get_starter_idx([r_ml, l_ml])
    starter = round(abs(starter_idx))
    corr_log = correlation_based_method(starter, sensor_data_filtered)

    #fft-based method
    fft_log = fft_method(starter, [r_ap, l_ap], len(sensor_data_filtered[0]))
    
    #Find places where logs agree
    final_log = find_log_overlap(corr_log, fft_log)
    
    #Merge FOGS with one or two second apart
    merged_log = merge_one_two_second(final_log)
    
    nn = merged_log.count(1)
    mm = len(merged_log)
    
    #Find elements where values changed
    nonzero_idxs = [i for i, x in enumerate(np.diff(merged_log)) if x != 0]
    
    #Find elements with minimal repetition
    logical_arr = [None] * len(nonzero_idxs)
    for idx, val in enumerate(nonzero_idxs):
        logical_arr[idx] = merged_log[val]
    dist_between_idxs = np.diff(nonzero_idxs)
    
    very_short = 0
    short = 0
    long = 0
    very_long = 0
    
    #record fogs
    for yes, idx in zip(logical_arr, dist_between_idxs.tolist()):
        if yes==1 and idx == 1:
            very_short = very_short + 1
        elif yes==1 and idx>=2 and idx<=5:
            short = short + 1
        elif yes==1 and idx > 5 and idx<=30:
            long = long + 1
        elif yes==1 and idx > 30:
            very_long = very_long + 1
    
    total = 100*((fog.get_nn() - fog.get_very_short())/fog.get_mm())
    
    fog_df = fog_df.append({'filename': filename,
                              'NN': nn,
                              'MM': mm,
                             'very short': very_short,
                             'short': short,
                             'long': long,
                             'very long': very_long,
                             'total': total}, ignore_index = True)
    x = x + 1
fog_df.to_excel(output_path)